# Tensorflow 的一个简单机器学习的例子

在此教程中，我们将用著名的MNIST数据集，去编写一个简单的logistic regression分类器，以用来识别手写数字图像。我们希望通过这个简单的例子进一步来说明 TensorFlow 工作流程。我们默认读者已经拥有一定的机器学习基础知识。



## MNIST 数据集

如果有对MNIST不了解的读者可以参考其[官网](http://yann.lecun.com/exdb/mnist/)。我们在这里调用一份现成的代码来读取这个数据集。首先[在这里](https://tensorflow.googlesource.com/tensorflow/+/master/tensorflow/examples/tutorials/mnist/input_data.py)下载读取代码，然后用下面的code将MNIST数据集读进Python.

In [6]:
import input_data

In [7]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 定义模型 -- Logistic regression

我们首先定义x，既我们的input data。 x不是一个特定的值，而是一个占位符placeholder，我们希望能够输入任意数量的MNIST图像，每一张图展平成784维的向量。因此，这个tensor的shape是[None，784]。（这里的None表示此张量的第一个维度可以是任何长度的。）

In [10]:
#  import tensorflow
import tensorflow as tf

# Define input data, which is a type of placehoder as float, with shape (None, 784). 
x = tf.placeholder("float", [None, 784])



然后我们定义我们的参数W和b。这里我们将它们定义为Variable。 Variable可以用于计算输入值，也可以在计算中被修改。对于各种机器学习应用，一般用Variable来表示模型参数。注意，variable在定义时需要同时定义初始值。初始值可以是任意数字，这里我们用一个truncated_normal distribution来初始W，用zero来初始b.

注意，W的维度是[784，10]，因为我们想要用784维的图片向量乘以它以得到一个10维的证据值向量，每一位对应不同数字类。b的形状是[10].


In [11]:
W = tf.Variable(tf.truncated_normal([784,10]))
b = tf.Variable(tf.zeros([10]))

现在，我们可以定义我们的logistic regression model。只需要一行代码！

In [12]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

## 定义loss和优化算法

每一个机器学习模型都需要定义loss函数，然后通过优化这个loss函数来训练模型，得到参数。Tensorflow提供了多种可用的loss函数，这里我们选取比较简单的cross-entropy函数作为我们的loss。

In [15]:
# y_ is your trainning data label
y_ = tf.placeholder("float", [None,10])

# define your cross entropy
cross_entropy = -tf.reduce_sum(y_*tf.log(y))


定义完loss函数之后，我们需要选择优化算法。Tensorflow已经为我们嵌入了各种常用的优化算法，简单从gradient descent到Adam。调用这些算法只需要一行代码！这里我们使用Gradient Descent。更多的优化算法可以[参考这里](https://www.tensorflow.org/api_guides/python/train#Optimizers)

In [17]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

## 训练模型

定义完优化算法之后，训练模型之签，我们需要初始化所有的我们的变量

In [18]:
init = tf.initialize_all_variables()

现在我们可以在一个 Session 里启动我们定义好的模型。

In [25]:
sess = tf.Session()
sess.run(init)

然后开始训练模型，这里我们让模型循环训练10000次！

In [29]:
for i in range(10000):
    # read data x and y, randomly select 100 data point -- stochastic gradient descent 
    batch_xs, batch_ys = mnist.train.next_batch(100)
    
    # feed them into model as x and y_
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

## 模型预测

训练好模型之后，我们开始用训练好的模型来预测testing data。首先我们需要定义测量模型好坏的指标，这里我们选用最直接的accuracy。可以看到，这么一个简单的logistic regresison模型的准确率是91%。


In [31]:
# tf.argmax is a very useful fucntion, which return the index with the lastest vaule. tf.equal returns a T/F.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

# change T/F to a binary value and calculate its mean -- accuracy.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# feed testing data into the trained model as x and y_; print the accuracy of testing data.
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.92


## 结语

这篇notebook为大家实现了一个基于Tensorflow的简单的logistic regression 分类器。虽然其简单，但是麻雀虽小五脏俱全。不管是基于Tensorflow的什么样复杂的模型，CNN或者RNN， 都遵循着这几个步骤：即定义模型，选择loss函数和优化算法，训练模型，最后用模型去预测。

UniDeep 数据科学家将为大家定期奉上Tensorflow和Keras的学习心得。我们会与大家一起从安装Tensorflow开始，系统地整理Tensorflow的学习笔记，并为大家整理Tensorflow在一些经典深度学习模型， 如：AlexNet， ResNet， LSTM上实现的代码。最后我们也会为大家介绍Keras， 这一基于Tensorflow框架的，简单方便的神经网络构建模块。

接下来，我们将继续使用MNIST数据集，为大家实现一个更为复杂的基于Tensorflow的分类器，敬请期待！